In [241]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

The idea is that an image contains more rain if there are less grey pixels.

In [243]:
df = pd.read_csv('id_df_SRI_jpg_radar_1.csv')
# Convert 'time' columns to time, format is hour:minute:second
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')

df['time_diff'] = df.iloc[::-1]['date_time'].diff()

print(len(df[df['time_diff'] < pd.Timedelta(minutes=11)]))

# add a new column "sequence" to divide the dataframe in sequences: 
# if the index is not consecutive, the sequence changes
sequence = 0
sequences = []
prev_idx = 0

for row in df[df['time_diff'] < pd.Timedelta(minutes=11)].itertuples():
    if row.Index != prev_idx+1:
        sequence += 1
    sequences.append(sequence)
    prev_idx = row.Index

df = None

18403


In [244]:
id_df = pd.read_csv("id_df_rain.csv")
id_df['sequence'] = sequences

In [245]:
print(id_df["not_rain_pxl"].describe(percentiles=[ .25, .5, .75, .9, .95, .99]))

count     18403.000000
mean     176643.576536
std       23786.576146
min       54082.000000
25%      168354.500000
50%      186743.000000
75%      193173.000000
90%      195111.600000
95%      195717.000000
99%      196148.000000
max      196158.000000
Name: not_rain_pxl, dtype: float64


In [246]:
# Define quantile thresholds
quantiles = {
    "0% - 25%": 168358,
    "25% - 50%": 186743,
    "50% - 75%": 193173,
    "75% - 95%": 195112,
    "95% - 100%": id_df["not_rain_pxl"].max()
}

# Create a new column 'rain_category' based on quantiles
id_df["rain_category"] = pd.cut(
    id_df["not_rain_pxl"],
    bins=[0, quantiles["0% - 25%"], quantiles["25% - 50%"], quantiles["50% - 75%"], quantiles["75% - 95%"], quantiles["95% - 100%"]],
    labels=[4, 3, 2, 1, 0],
    include_lowest=True
)

id_df["rain_category"] = id_df["rain_category"].astype(int)

In [247]:
print(id_df)

            id            date_time  not_rain_pxl  sequence  rain_category
0      2457769  2024-10-28 09:10:00        196104         1              0
1      2455220  2024-10-26 14:00:00        195786         2              0
2      2455200  2024-10-26 13:50:00        195662         2              0
3      2455180  2024-10-26 13:40:00        195610         2              0
4      2455162  2024-10-26 13:30:00        195463         2              0
5      2455140  2024-10-26 13:20:00        195425         2              0
6      2455120  2024-10-26 13:10:00        195518         2              0
7      2455080  2024-10-26 13:00:00        195536         2              0
8      2455060  2024-10-26 12:50:00        195138         2              0
9      2455040  2024-10-26 12:40:00        194505         2              1
10     2455020  2024-10-26 12:30:00        194766         2              1
11     2455002  2024-10-26 12:20:00        194610         2              1
12     2454980  2024-10-2

In [248]:
seq_len = id_df.groupby('sequence').size()

id_df = id_df[id_df['sequence'].map(seq_len) > 10]

seq_rain = id_df.groupby('sequence')['rain_category'].mean()
seq_rain = seq_rain[seq_rain > 0.1]
id_df = id_df[id_df['sequence'].isin(seq_rain.index)]

# reorder the number of the sequences: if there is a missing number, fill it with the previous number +1
id_df['sequence'] = id_df['sequence'].rank(method='dense').astype(int)
id_df['sequence'] = id_df['sequence'] - 1

print(len(id_df))

17669


In [249]:
id_df.to_csv("id_df_final.csv", index=False)